In [ ]:
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Pillow
from PIL import Image
# Torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import pandas as pd

In [ ]:
from dataloaders import *
from model_classes import *

In [ ]:
ld_val = Lung_Val_Dataset1()
ld_test = Lung_Test_Dataset1()
ld_train = Lung_Train_Dataset1()

ld_val2 = Lung_Val_Dataset2()
ld_test2 = Lung_Test_Dataset2()
ld_train2 = Lung_Train_Dataset2()

ld_val3 = Lung_Val_Dataset3()
ld_test3 = Lung_Test_Dataset3()
ld_train3 = Lung_Train_Dataset3()

In [ ]:
bs_val = 32 #batch size

In [ ]:
# Dataloader from dataset
train_loader = DataLoader(ld_train, batch_size = bs_val, shuffle = True)
test_loader = DataLoader(ld_test, batch_size = bs_val, shuffle = True)
val_loader = DataLoader(ld_val, batch_size = bs_val, shuffle = True)

# Dataloader from dataset2
train_loader2 = DataLoader(ld_train2, batch_size = bs_val, shuffle = True)
test_loader2 = DataLoader(ld_test2, batch_size = bs_val, shuffle = True)
val_loader2 = DataLoader(ld_val2, batch_size = bs_val, shuffle = True)

# Dataloader from dataset3
train_loader3 = DataLoader(ld_train3, batch_size = bs_val, shuffle = True)
test_loader3 = DataLoader(ld_test3, batch_size = bs_val, shuffle = True)
val_loader3 = DataLoader(ld_val3, batch_size = bs_val, shuffle = True)

In [ ]:
#hyper parameters:
n_epochs = 10
lr = 0.001

In [ ]:
# Create model
model = Net2()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [ ]:
# Define validation function 
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    
    for images, labels in testloader:
        output = model.forward(images)
        test_loss += criterion(output, torch.max(labels, 1)[1]).item()
        
        ps = torch.exp(output)
        equality = (torch.max(labels, 1)[1].data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()

    return test_loss, accuracy

In [ ]:
# set model name
model_name = "model_new"

In [ ]:
running_loss = 0

train_losses = []
val_losses = []
accuracies = []

for e in range(n_epochs):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader3):
        #print(batch_idx)
        #images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        output = model.forward(images)
        #loss = criterion(output, labels)
        loss = criterion(output, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Eval mode for predictions
    model.eval()

    # Turn off gradients for validation
    with torch.no_grad():
        test_loss, accuracy = validation(model, test_loader3, criterion, 1)

    accur = accuracy.item()/len(test_loader3)
    train_loss = running_loss/bs_val
    val_loss = test_loss/len(test_loader3)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    accuracies.append(accur)

    print("Training Loss is " + str(train_loss) + " | Test Loss is: " + str(val_loss) + " | Accuracy is: " + str(accur))
    print("\n")
    train_losses.append(running_loss/bs_val)
    val_losses.append(test_loss/len(test_loader3))
    running_loss = 0

    # Make sure training is back on
    model.train()

    #save the current epoch model.
    torch.save({
            'epoch': n_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, "models/" + model_name + "_epoch" + str(e+1) + ".pth")

In [ ]:
#save the train loss, test loss and accuracy as a csv file
df = pd.DataFrame()
df["train_loss"] = train_losses
df["test_loss"] = val_losses
df["accuracy"] = accuracies

df.to_excel("training_logs/" + model_name + ".xlsx")